In [ ]:
# default_exp pipeline

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from counternet.import_essentials import *
from counternet.utils.all import *
from counternet.training_module import *

Global seed set to 31


## Utils

In [ ]:
# export
def load_trained_model(module: BaseModule, checkpoint_path: str, gpus : int = 0) -> BaseModule:
    # assuming checkpoint_path = f"{dict_path}/epoch={n_epoch}-step={step}.ckpt"
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f'{checkpoint_path} is not found.')

    n_iter = int(checkpoint_path.split("-")[0].split("=")[-1]) + 1
    model = module.load_from_checkpoint(checkpoint_path)
    tmp_trainer = pl.Trainer(
        max_epochs=n_iter, resume_from_checkpoint=checkpoint_path, num_sanity_val_steps=0, gpus=gpus,
        logger=False, checkpoint_callback=False
    )
    tmp_trainer.fit(model)

    return model

## Trainer

In [ ]:
# export
class CFNetTrainer(object):
    def __init__(self,
                 model: BaseModule,
                 t_configs: Dict[str, Any],
                 callbacks: Optional[List[Callback]] = None,
                 description: Optional[str] = None,
                 debug: Optional[bool] = False,
                 logger: Optional[LightningLoggerBase] = None,
                 logger_name: str = "debug"):

        if logger is None:
            logger = pl_loggers.TestTubeLogger(
                Path('../log/'),
                name=logger_name,
                description=description,
                debug=debug,
                log_graph=True
            )

        # model checkpoint
        checkpoint_callback = ModelCheckpoint(
            monitor='val/val_loss',
            save_top_k=3,
            mode='min'
        )

        # define callbacks
        if callbacks is None:
            callbacks = [checkpoint_callback]
        elif self._has_no_model_checkpoint(callbacks):
            callbacks += [checkpoint_callback]

        self.trainer = pl.Trainer(
            logger=logger,
            callbacks=callbacks,
            **t_configs
        )

        self.model = model

    def _has_no_model_checkpoint(self, callbacks: List[Callback]) -> bool:
        for callback in callbacks:
            if isinstance(callback, ModelCheckpoint):
                return False
        return True

    def fit(self, is_parallel=False):
        if is_parallel:
            logging.warning(
                f"parallel version has not been implemented\nUsing the single process training...")
        self.trainer.fit(self.model)

        return dict(
            trainer=self.trainer,
            module=self.model
        )

    def load_trained_model(self, checkpoint_path: str, gpus: int = 0) -> BaseModule:
        self.model = load_trained_model(
            self.model, checkpoint_path=checkpoint_path, gpus=gpus)
        return self.model

In [ ]:
t_config = load_json('assets/configs/trainer.json')
m_config = load_json('assets/configs/adult.json')
# model = 

## CF Generator
- local explainer
- global explainer